In [1]:
%load_ext autoreload
import os
from functools import partial
from dataclasses import dataclass
from typing import Optional, Callable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from pmf import PoissonMF

# Load & Process Data

In [2]:
data_path = './data/'
notes = pd.read_csv(os.path.join(data_path, 'notes-00000.tsv'), sep='\t')
# Convert NaN to empty string
notes['summary'] = notes['summary'].astype(str).fillna('').str.strip()

ratings = pd.read_csv(os.path.join(data_path, 'ratings-00000.tsv'), sep='\t')

# Drop rows with NaN in helpfulnessLevel column
ratings = ratings.dropna(subset=['helpfulnessLevel'])

In [3]:
# Print total number of ratings
print('Total number of ratings: {}'.format(len(ratings)))

# Print number of unique notes and raters
print('Number of unique notes: {}'.format(ratings['noteId'].nunique()))
print('Number of unique raters: {}'.format(ratings['raterParticipantId'].nunique()))

# Get list of notes with more than 5 ratings
note_rating_counts = ratings['noteId'].value_counts()
filtered_note_ids = note_rating_counts[note_rating_counts > 5].index.tolist()
print('Number of notes with more than 5 ratings: {}'.format(len(notes)))

# Get list of raters with more than 10 ratings
rater_counts = ratings['raterParticipantId'].value_counts()
filtered_rater_ids = rater_counts[rater_counts > 10].index.tolist()
print('Number of raters with more than 10 ratings: {}'.format(len(filtered_rater_ids)))

# Filter ratings to only include ratings rated by raters with more than 10 ratings and for notes with more than 5 ratings
ratings = ratings[ratings['raterParticipantId'].isin(filtered_rater_ids) & ratings['noteId'].isin(filtered_note_ids)]
print('Number of ratings after filtering: {}'.format(len(ratings)))

Total number of ratings: 5347044
Number of unique notes: 113682
Number of unique raters: 131866
Number of notes with more than 5 ratings: 122641
Number of raters with more than 10 ratings: 54350
Number of ratings after filtering: 5003405


In [4]:
# Convert the ratings matrix to three lists:
# - rating_labels, which is the 'helpfulnessLevel' column mapped to -1 for 'NOT_HELPFUL',
#   0 for 'SOMEWHAT_HELPFUL', and 1 for 'HELPFUL'
# - user_idxs, which is the 'raterParticipantId' column mapped to a unique integer
# - note_idxs, which is the 'noteId' column mapped to a unique integer
rating_labels = ratings['helpfulnessLevel'].map({'NOT_HELPFUL': -1, 'SOMEWHAT_HELPFUL': 0, 'HELPFUL': 1})
# Use a label encoder to map the user and note ids to unique integers
user_encoder = LabelEncoder()
note_encoder = LabelEncoder()
user_idxs = user_encoder.fit_transform(ratings['raterParticipantId'])
note_idxs = note_encoder.fit_transform(ratings['noteId'])

n_users = len(user_encoder.classes_)
n_notes = len(note_encoder.classes_)

# Sparse exposure matrix (did the user rate the note?)
exp_matrix = csr_matrix((np.ones_like(rating_labels), (user_idxs, note_idxs)), shape=(n_users, n_notes))

In [5]:
n_notes, n_users

(89064, 54350)

# Step 1a: Causal Inference, Exposure Model
Fit Poisson matrix factorization to the exposures/assignments (who rated what). We will then use the reconstructed exposures as substitute confounders.

In [6]:
pf = PoissonMF(n_components=1, random_state=1, verbose=True, a=0.3, b=0.3, c=0.3, d=0.3)
pf.fit(exp_matrix, user_idxs, note_idxs)

PoissonMF(n_components=1, random_state=1, verbose=True)

In [7]:
# Latent representations learned by Poisson MF
exp_user_factors, exp_item_factors = pf.Eb, pf.Et.T

# Step 1b: Causal Inference, Outcome Model
Now estimate the outcome model, i.e., matrix factorization on the observed ratings while controlling for the substitute confounders estimated from Step 1a.

In [8]:
%load_ext autoreload
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch import nn
from mf import MatrixFactorizationModel, ModelData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
mf_model = MatrixFactorizationModel(n_users, n_notes, exp_user_factors=exp_user_factors, exp_item_factors=exp_item_factors, n_components=4)

rating_tensor = torch.FloatTensor(rating_labels).to(mf_model.device)
user_idxs_tensor = torch.LongTensor(user_idxs).to(mf_model.device)
note_idxs_tensor = torch.LongTensor(note_idxs).to(mf_model.device)
exp_tensor = torch.ones_like(rating_tensor).to(mf_model.device)

data = ModelData(rating_tensor, user_idxs_tensor, note_idxs_tensor, exp_tensor)

In [40]:
train_loss, val_loss = mf_model.fit(data, epochs=150, lr=0.1, print_interval=20, validate_fraction=0.1, print_loss=True)

Epoch 0: train L2-reg loss = 0.465, val L2-reg loss = 0.899
Epoch 0: train MSE = 0.898, val MSE = 0.897
Epoch 20: train L2-reg loss = 0.221, val L2-reg loss = 0.462
Epoch 20: train MSE = 0.356, val MSE = 0.425
Epoch 40: train L2-reg loss = 0.191, val L2-reg loss = 0.411
Epoch 40: train MSE = 0.298, val MSE = 0.370
Epoch 60: train L2-reg loss = 0.186, val L2-reg loss = 0.401
Epoch 60: train MSE = 0.291, val MSE = 0.361
Epoch 80: train L2-reg loss = 0.186, val L2-reg loss = 0.400
Epoch 80: train MSE = 0.290, val MSE = 0.359
Epoch 100: train L2-reg loss = 0.186, val L2-reg loss = 0.399
Epoch 100: train MSE = 0.290, val MSE = 0.359
Epoch 120: train L2-reg loss = 0.185, val L2-reg loss = 0.399
Epoch 120: train MSE = 0.290, val MSE = 0.358
Epoch 140: train L2-reg loss = 0.185, val L2-reg loss = 0.399
Epoch 140: train MSE = 0.290, val MSE = 0.358


# Step 2: Voting Aggregation
Calculate results for different voting aggregation rules.

In [41]:
pd.set_option('display.max_colwidth', 1000)

In [42]:
filtered_notes = notes[notes['noteId'].isin(filtered_note_ids)]
note_ids = note_encoder.inverse_transform(np.arange(n_notes))

## Mean

In [43]:
majority_votes = mf_model.forward_majority_vote()

In [44]:
note_results = pd.DataFrame({'noteId': note_ids, 'noteScore': majority_votes})
scored_notes = filtered_notes.merge(note_results, on='noteId')
scored_notes = scored_notes.sort_values(by='noteScore', ascending=False)
scored_notes.head(20)[['noteId', 'summary', 'noteScore']]

,noteId,summary,noteScore
60931,1595466720113836034,Ms Grossman has been charged with killing 2 children while racing her car while intoxicated. https://www.lamag.com/article/exclusive-rebecca-grossman-speaks-out-ahead-of-murder-trial/,0.762114
14959,1651670631677607938,An animated Transformers movie came out in 1986 https://www.imdb.com/title/tt0092106/,0.756467
50618,1651712018674483200,"The Act of emulation is not a crime as noted in the case of Sony Computer Entertainment America, Inc. v. Bleem, LLC. https://h2o.law.harvard.edu/cases/5198 The only illegal part of emulation is distributing copyrighted software and the BIOS files from consoles, the emulators themselves are not illegal.",0.753539
58370,1651791523552395266,Cannons are lethal. https://www.civilwarmed.org/effects-of-artillery/,0.736728
70429,1656462834560303104,This video is from 6 years ago (2017). It has nothing to do with Covid or vaccinations. Here is the link to the original video: https://www.youtube.com/watch?v=QpvGym2w8F8,0.724251
28461,1648877504197804037,"These payment estimates are deceptively low. A $50,000 loan at 1.5% interest and payments of $241/month would take 20 years to pay. https://www.bankrate.com/loans/loan-calculator/ Typical HELOC rates as of April 2023 are 7-8%. At 7% interest a $50k loan with $241 payments would take longer than 50 years to pay. https://www.forbes.com/advisor/home-equity/best-heloc-rates/",0.722635
45274,1646715834096844800,"These payment estimates are deceptively low. A $50,000 loan at 1.5% interest and payments of $241/month would take 20 years to pay. https://www.bankrate.com/loans/loan-calculator/ Typical HELOC rates as of April 2023 are 7-8%. At 7% interest a $50k loan with $241 payments would take longer than 50 years to pay. https://www.forbes.com/advisor/home-equity/best-heloc-rates/",0.720561
76997,1666860708942819330,The company above is using stolen video from the Autel Company and are using it to scam buyers by making them think they are getting a high quality drone for an unreasonably low price. The same type scam is used constantly on Facebook. @elonmusk needs to add scam reporting https://www.autelrobotics.com/productdetail/evo-lite-series-drones.html,0.719693
25957,1640836719783170050,"This headline comes from SpaceXMania, a satire news site. https://spacexmania.com/elon-musk-sues-the-view-and-whoopi-goldberg-for-60-million-claims-they-are-lying-about-him/",0.719426
73638,1652007736773865473,"The Earth is round. Other Nikon super zoom cameras have been used to demonstrate this fact both by observing the curve occluding objects and by imaging satellites on orbit, which is only possible with a round earth. Orbiting spacecraft: https://youtu.be/hqr_qbW_avo Occluded objects: https://youtu.be/y8MboQzXO1o",0.719195


## Quantile

In [45]:
quantile = partial(torch.quantile, q=0.25)
quantile_scores = mf_model.get_vote_scores(quantile)

In [55]:
note_results = pd.DataFrame({'noteId': note_ids, 'noteScore': quantile_scores})
scored_notes = filtered_notes.merge(note_results, on='noteId')
scored_notes = scored_notes.sort_values(by='noteScore', ascending=False)
scored_notes.head(20)[['noteId', 'summary', 'noteScore']]

,noteId,summary,noteScore
60931,1595466720113836034,Ms Grossman has been charged with killing 2 children while racing her car while intoxicated. https://www.lamag.com/article/exclusive-rebecca-grossman-speaks-out-ahead-of-murder-trial/,0.550131
14959,1651670631677607938,An animated Transformers movie came out in 1986 https://www.imdb.com/title/tt0092106/,0.548730
50618,1651712018674483200,"The Act of emulation is not a crime as noted in the case of Sony Computer Entertainment America, Inc. v. Bleem, LLC. https://h2o.law.harvard.edu/cases/5198 The only illegal part of emulation is distributing copyrighted software and the BIOS files from consoles, the emulators themselves are not illegal.",0.533234
58370,1651791523552395266,Cannons are lethal. https://www.civilwarmed.org/effects-of-artillery/,0.527377
70429,1656462834560303104,This video is from 6 years ago (2017). It has nothing to do with Covid or vaccinations. Here is the link to the original video: https://www.youtube.com/watch?v=QpvGym2w8F8,0.522178
25957,1640836719783170050,"This headline comes from SpaceXMania, a satire news site. https://spacexmania.com/elon-musk-sues-the-view-and-whoopi-goldberg-for-60-million-claims-they-are-lying-about-him/",0.520633
73638,1652007736773865473,"The Earth is round. Other Nikon super zoom cameras have been used to demonstrate this fact both by observing the curve occluding objects and by imaging satellites on orbit, which is only possible with a round earth. Orbiting spacecraft: https://youtu.be/hqr_qbW_avo Occluded objects: https://youtu.be/y8MboQzXO1o",0.519052
45274,1646715834096844800,"These payment estimates are deceptively low. A $50,000 loan at 1.5% interest and payments of $241/month would take 20 years to pay. https://www.bankrate.com/loans/loan-calculator/ Typical HELOC rates as of April 2023 are 7-8%. At 7% interest a $50k loan with $241 payments would take longer than 50 years to pay. https://www.forbes.com/advisor/home-equity/best-heloc-rates/",0.518585
73036,1653431562489286657,"This is not MH370. MH370 has never been found as of 2023, this photo is a picture of a underwater Lockheed Martin L1011 Tristar that was abandoned at an airport and sunken to attract diving tourism. https://www.cnn.com/travel/article/red-sea-underwater-abandoned-plane/index.html",0.518149
28461,1648877504197804037,"These payment estimates are deceptively low. A $50,000 loan at 1.5% interest and payments of $241/month would take 20 years to pay. https://www.bankrate.com/loans/loan-calculator/ Typical HELOC rates as of April 2023 are 7-8%. At 7% interest a $50k loan with $241 payments would take longer than 50 years to pay. https://www.forbes.com/advisor/home-equity/best-heloc-rates/",0.518118


## Min

In [47]:
def min2(x, dim):
  return torch.min(x, dim=dim).values

min_scores = mf_model.get_vote_scores(min2)

In [48]:
note_results = pd.DataFrame({'noteId': note_ids, 'noteScore': min_scores})
scored_notes = filtered_notes.merge(note_results, on='noteId')
scored_notes = scored_notes.sort_values(by='noteScore', ascending=False)
scored_notes.head(20)[['noteId', 'summary', 'noteScore']]

,noteId,summary,noteScore
67439,1654264640182861824,Is a fake photo,0.001321
64377,1536319775609085952,https://www.foxnews.com/politics/biden-us-ukraine-rocket-systems-reach-russia This Is from Fox News.,0.000166
75772,1637922022775828482,"A postagem a seguir descumpre com a comunidade. A lei 6015/73 apenas proíbe nomes &quot;suscetíveis de expor ao ridículo os seus portadores&quot;, ficando a cargo do oficial cartorário apontar isso em cada caso. A lista referida é de nomes que o cadastro eletrônico do SUS proíbe.",-0.001522
68216,1666461666811281409,宮島に半世紀以上住んでる者として言わせて貰います。 廿日市市の人へと言うのがそもそもの間違い。 鹿の人への漁り問題は廿日市市と宮島町が比較的最近合併するより遥か前から起きており、宮島町住人以外の廿日市市民は実情すら知らない。 そもそも鹿は雑食で、何でも手当たり次第食べる。 知らない人が不用意に食べ物を与えるから、人間の持ち物や食べ物を狙ってしまう。 その為、宮島の民家の玄関は鹿が入らないよう引き戸が多い。 本来なら、下手な餌付けはしないで宮島の大半の面積を占める山で暮らさせるべき。 このツイートの勘違いは、全ての点で本当に迷惑です。,-0.074106
63058,1545622719647174656,The force required to break a human skull is around 2000N. A 7.62x39 round fired straight up that reaches terminal velocity the bullet would impact with 700N of force. Not enough to break the skull of an average adult. Falling bullets are not lethal under normal circumstances.,-0.086303
69078,1642347795800551426,No company is offering mRNA animal vaccines on the open market. No drug company has laid claim to be to producing it or put methodologies in place that would allow for mass production. Animal pharmaceutical companies are just getting money to explore the new technology https://www.alderspring.com/organic-beef-matters/glenns-response-to-the-mrna-vaccine/ https://www.factcheck.org/2021/07/scicheck-meme-spreads-falsehood-about-vaccine-transfer/,-0.111412
54010,1557192174353137664,"Between August 5th and August 9th, Musk sold 7.92 million Tesla shares worth $6.9 billion, according to SEC filings. https://www.reuters.com/business/autos-transportation/elon-musk-sells-792-million-tesla-shares-worth-69-billion-sec-filing-2022-08-10/",-0.131657
60310,1585046043531649025,This Winter Wondelanc drive is not in Arizona it’s a whole other country amd that country is Greece 🇬🇷. Sometimes we believe everything on the internet but with a little research we find the truth https://greekreporter.com/2022/01/23/winter-wonderland-video-greece/,-0.134689
42145,1467373430043561984,Huge political opponents have completely opposite evaluations of her. This was visible during the Senate Hearings. https://www.buzzfeednews.com/article/emmaloop/kamala-harris-intelligence-committee-russia-republicans-2020,-0.139242
45334,1659194460847353857,Ja Morant merchandise is still available on Nike.com https://nike.sng.link/Astn5/6tbz/r_480c9299f2 Along with his sneakers still set to release May 25 on the Nike SNKRS app. https://snkrs.sng.link/Aoihh/9p4r/r_b68f6952e5,-0.140117


## Max

In [49]:
def max2(x, dim):
  return torch.max(x, dim=dim).values

max_scores = mf_model.get_vote_scores(max2)

In [50]:
note_results = pd.DataFrame({'noteId': note_ids, 'noteScore': max_scores})
scored_notes = filtered_notes.merge(note_results, on='noteId')
scored_notes = scored_notes.sort_values(by='noteScore', ascending=False)
scored_notes.head(20)[['noteId', 'summary', 'noteScore']]

,noteId,summary,noteScore
51119,1650377087457296384,This tweet is part of an ongoing gag that is an attempt to keep people from consuming oxtails. https://blackamericaweb.com/playlist/oxtails-black-twitter/ Oxtails are safe to eat in moderation when prepared properly. https://www.msn.com/en-us/health/nutrition/downsides-of-oxtail-see-what-the-experts-say/ar-AA14vvP0,5.307672
69334,1656789686344450052,None of these companies have given out the listed scores for The Legend of Zelda: Tears of the Kingdom https://www.ign.com/articles/the-legend-of-zelda-tears-of-the-kingdom-review https://www.destructoid.com/reviews/review-the-legend-of-zelda-tears-of-the-kingdom/ https://www.gamespot.com/reviews/the-legend-of-zelda-tears-of-the-kingdom-review/1900-6418063/ https://www.godisageek.com/reviews/the-legend-of-zelda-tears-of-the-kingdom-review/ https://www.nintendolife.com/reviews/nintendo-switch/the-legend-of-zelda-tears-of-the-kingdom https://www.rpgsite.net/review/14176-the-legend-of-zelda-tears-of-the-kingdom-review https://screenrant.com/zelda-tears-kingdom-totk-review-switch-performance/?utm_content=buffer964e6&amp;utm_medium=Social-Distribution&amp;utm_source=SR-TW&amp;utm_campaign=SR-TW https://www.gameinformer.com/review/the-legend-of-zelda-tears-of-the-kingdom/such-great-heights https://www.videogameschronicle.com/review/zelda-tears-of-the-kingdom/ https://www.vg24...,4.778496
62970,1670170815491461120,It's a meme NNN,3.921623
19929,1671446277777768448,"NNN, responses to the question need to go in the comments And just in case this is obviously an edited image",3.908300
73707,1670200892685942784,"Very big NNN, even if the post wasn't a joke, the 2 countries are very small and can be esaily not mencionated",3.877073
38876,1648089046831312899,No es necesaria añadir una nota. El tweet muestra imágenes de la historia del Barcelona.,3.847260
56713,1668736637281394690,"Claude été le gagnant de Koh-Lanta en 2021. Ce compte est réputé pour donné des informations réelle mais avec du retard. Lien pour en savoir + sur Claude. https://fr.m.wikipedia.org/wiki/Claude_Dartois#:~:text=En%202021%2C%20Claude%20annonce%20participer,les%20%C3%A9preuves%20sur%20Koh%2DLanta.",3.845274
20987,1668920383779504136,"Le Gorafi is een satirewebsite, er valt dus niets te weerleggen. https://fr.wikipedia.org/wiki/Le_Gorafi",3.804518
70519,1647716282744053760,"Opinions do not require Community Notes, if you want to argue use the reply function.",3.738807
11367,1656789871321808897,There is no reason for a CN on this.,3.738398


## Controversial (Variance)

In [51]:
var_scores = mf_model.get_vote_scores(torch.var)

In [52]:
note_results = pd.DataFrame({'noteId': note_ids, 'noteScore': var_scores})
scored_notes = filtered_notes.merge(note_results, on='noteId')
scored_notes = scored_notes.sort_values(by='noteScore', ascending=False)
scored_notes.head(20)[['noteId', 'summary', 'noteScore']]

,noteId,summary,noteScore
73599,1671564596518191132,« Les Soulèvements de la Terre » est un collectif écologiste qui prône l’insurrection. Le mouvement est accusé d’être à l’origine d'actions violentes. Cette dissolution est une réponse pénale à l'égard des troubles graves à l'ordre qui eu lieu à Sainte-Soline : https://www.nouvelobs.com/ecologie/20230329.OBS71495/qui-sont-les-soulevements-de-la-terre-le-mouvement-que-veut-dissoudre-darmanin.amp,0.626156
44934,1669986609662443520,"Manche sollten Kollektive Anmerkungen nicht für die Untermalung ihrer persönlichen Weltsicht missbrauchen. Tweet/ Artikel geht konkret auf eine Falschdarstellung ein, setzt sie in den Kontext und belegt es. Kein Kommentar notwendig. Plus: Correctiv sind seriös, arbeiten sauber.",0.551492
64814,1672418674278567937,"Biológicamente existe hombre o mujer. El género que una persona decida tener con base a su percepción es una decisión individual, la cual es respetable, sin embargo la percepción de otros es algo que no puede imponerse.",0.542932
64822,1663705081445855236,La foto está alterada digitalmente para hacer parecer que los ojos de la actriz luzcan más separados de lo que realmente están en los trailers https://youtu.be/kpGo2_d3oYE https://youtu.be/HrjIOgH6IiI,0.539244
70480,1668242509607100425,"Sim, crianças transgênero existem. https://www.sciencedirect.com/science/article/abs/pii/S0031395515000656?via%3Dihub Estudos mostram que a liberdade para expressar seu gênero como quiser desde a infância reduz consideravelmente o sofrimento mental de pessoas trans. Sofrimento causado pelo preconceito e ostracismo social enfrentados. https://doi.org/10.1590/1518-8345.3504.3351",0.514411
64100,1658666704849739778,"https://www.tse.jus.br/comunicacao/noticias/2023/Maio/por-unanimidade-tse-cassa-registro-do-deputado-federal-deltan-dallagnol-pode A decisão pautada na lei ficha limpa que determinou a cassação do mandado de Deltan Dallagnol foi feita em unanimidade pelos ministros do TSE, não se trata de uma canetada como ele alega.",0.508275
73316,1670478147249405952,"Percentage of children living absent their biological fathers: Black children: 64% Hispanic children: 42% White children: 24% Asian children: 16% https://datacenter.aecf.org/data/tables/107-children-in-single-parent-families-by-race-and-ethnicity#detailed/1/any/false/2048,1729,37,871,870,573,869,36,868,867/10,11,9,12,1,185,13/432,431",0.505454
8451,1648456567472402434,マイナンバーカードの記載情報を求めているが、マイナンバーカードの裏面記載の個人番号は個人情報保護法で厳重取扱が求められているにも関わらず、その旨が記載されていない https://sumoviva.jp/article/1000890,0.498993
44387,1672599133339672577,Obama’s order 13603 does not involve the possibility to suspend the US Constitution. The order by Obama was a minor update of a similar order by President Clinton. The Defense Production Act has been in effect since the Truman Administration &amp; amended several times since https://www.snopes.com/fact-check/national-defense-resources-preparedness/ https://www.presidency.ucsb.edu/documents/executive-order-13603-national-defense-resources-preparedness,0.495927
30023,1669024656014155799,Gender affirming care is the standard of care supported by every major medical association in the United States and is recognized as the gold standard for treating gender dysphoria in trans youth. https://www.aamc.org/news/what-gender-affirming-care-your-questions-answered https://www.newswise.com/articles/ama-strengthens-its-policy-on-protecting-access-to-gender-affirming-care Gender-Affirming Care and Young People - HHS Office of Population Affairs https://opa.hhs.gov/sites/default/files/2022-03/gender-affirming-care-young-people-march-2022.pdf,0.494037


## % approval
Rank by the % approval (where approval is defined as a score being above a particular threshold)

In [53]:
def approval(x, dim, threshold=0.7):
    return (x > threshold).float().mean(dim=dim)

approval_scores = mf_model.get_vote_scores(approval)

In [54]:
note_results = pd.DataFrame({'noteId': note_ids, 'noteScore': approval_scores})
scored_notes = filtered_notes.merge(note_results, on='noteId')
scored_notes = scored_notes.sort_values(by='noteScore', ascending=False)
scored_notes.head(20)[['noteId', 'summary', 'noteScore']]

,noteId,summary,noteScore
60931,1595466720113836034,Ms Grossman has been charged with killing 2 children while racing her car while intoxicated. https://www.lamag.com/article/exclusive-rebecca-grossman-speaks-out-ahead-of-murder-trial/,0.625428
50618,1651712018674483200,"The Act of emulation is not a crime as noted in the case of Sony Computer Entertainment America, Inc. v. Bleem, LLC. https://h2o.law.harvard.edu/cases/5198 The only illegal part of emulation is distributing copyrighted software and the BIOS files from consoles, the emulators themselves are not illegal.",0.619834
14959,1651670631677607938,An animated Transformers movie came out in 1986 https://www.imdb.com/title/tt0092106/,0.617810
58370,1651791523552395266,Cannons are lethal. https://www.civilwarmed.org/effects-of-artillery/,0.598970
70429,1656462834560303104,This video is from 6 years ago (2017). It has nothing to do with Covid or vaccinations. Here is the link to the original video: https://www.youtube.com/watch?v=QpvGym2w8F8,0.587397
73638,1652007736773865473,"The Earth is round. Other Nikon super zoom cameras have been used to demonstrate this fact both by observing the curve occluding objects and by imaging satellites on orbit, which is only possible with a round earth. Orbiting spacecraft: https://youtu.be/hqr_qbW_avo Occluded objects: https://youtu.be/y8MboQzXO1o",0.586403
25957,1640836719783170050,"This headline comes from SpaceXMania, a satire news site. https://spacexmania.com/elon-musk-sues-the-view-and-whoopi-goldberg-for-60-million-claims-they-are-lying-about-him/",0.584213
28461,1648877504197804037,"These payment estimates are deceptively low. A $50,000 loan at 1.5% interest and payments of $241/month would take 20 years to pay. https://www.bankrate.com/loans/loan-calculator/ Typical HELOC rates as of April 2023 are 7-8%. At 7% interest a $50k loan with $241 payments would take longer than 50 years to pay. https://www.forbes.com/advisor/home-equity/best-heloc-rates/",0.584029
46610,1654890921266397184,"The glass used for blender cups is tempered and highly resistant to shattering, the action shown here is dangerous as the average mason jar is not built to withstand the speed and force of blender blades. https://www.thedonutwhole.com/can-you-use-mason-jars-for-smoothies/",0.583514
2984,1647666096613339136,This image as been digitally altered. The original can be found here: https://nymag.com/intelligencer/2015/10/lindsey-graham-pours-one-out-for-his-candidacy.html,0.581435
